In [1]:
# Multiple outputs per cell in Jupyter 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%load_ext jupyternotify
%autonotify -a 30

<IPython.core.display.Javascript object>

In [2]:
%%javascript
// Evitar autoscroll.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
from integrado import *
%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
tokens=Tokens("data/geo-mx-2004_min.json",replace=True,showProgressEach=10000)

** Processing data/geo-mx-2004_min.json ...
2020-05-12 20:54:48.410406 :: 0.000294 - Processing item #0 
** Processed 1000 lines. Saved to /home/jovyan/public/B6_ProyectoIntegrador/data/geo-mx-2004_min.npy.


In [ ]:
index_=Index(tokens,showProgressEach=10000)
len(index_.postlists.keys())

In [ ]:
ds=index_.o("covid coronavirus pandemia cuarentena",tokenize=True)
print(len(ds),ds[:5])
docs=tokens.getToks(ds)
print(len(docs),docs[:5])
[tw['text'] for tw in tokens.getTws(ds[:5])]

In [5]:
from gensim.models import FastText

In [6]:
class Embeddings():
    
    def __init__(self, tokens, showProgressEach=1000):
        print("Init...")
        model = FastText(size=100, window=3, min_count=1, min_n=2, max_n=5)
        print("Build...")
        model.build_vocab(sentences=tokens)
        print("Train...")
        model.train(sentences=tokens,total_examples=len(tokens), epochs=10)
        print("Mean...")        
        arry=[]
        tx = datetime.datetime.now()
        for idx, tw in enumerate(tokens):
            tb.show_progress(showProgressEach,tx,idx)
            arry.append(np.mean([model.wv[tok] for tok in tw],axis=0))
        self.embeddings_=np.array(arry)
#         self.embeddings_=np.array(
#             [np.mean([model.wv[tok] for tok in tw],axis=0) 
#                    for tw in tokens])
        del(model)
        gc.collect()

In [ ]:
tokens.pointer[:,:]

In [9]:
# tokens.pointer[:,:]
embs=Embeddings(tokens.pointer[:,:])
print(type(embs), len(embs))
embs

Init...
Build...
Train...


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/gensim/models/base_any2vec.py", line 211, in _worker_loop
    tally, raw_tally = self._do_train_job(data_iterable, job_parameters, thread_private_mem)
  File "/opt/conda/lib/python3.7/site-packages/gensim/models/fasttext.py", line 836, in _do_train_job
    tally += train_batch_cbow(self, sentences, alpha, work, neu1)
  File "gensim/models/fasttext_inner.pyx", line 696, in gensim.models.fasttext_inner.train_batch_cbow
  File "gensim/models/fasttext_inner.pyx", line 525, in gensim.models.fasttext_inner.populate_ft_config
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()



KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [ ]:
from skeleton import Clustering
class Clustering2(Clustering):
    
    # se asigna el método de inicialización de Kmeans (default="fft")
    def __init__(self, *args, k_init="fft", **kwargs):
        super().__init__(*args, **kwargs)
        self.k_init_ = k_init
        
    # asigna los elementos en la colección a su centroide más cercano
    # genera las etiquetas de los clusters
    # almacena la distancia de cada elemento a su centroide más cercano
    def _assign_nearest_centroids(self):
        self.labels_=[-1 for _ in self.data]
        self.dists_=[-1 for _ in self.data]
        for j in range(len(self.data)):
            dists = [(self.distance_function(c, self.data[j]), i, self.data[j])
                     for i, c in self.centroids_.items()]
            dists.sort()
            self.labels_[j]=dists[0][1]
            self.dists_[j]=(dists[0][0],j)
            
    def KMeans(self):
        self.centroids_ = {}
        C = []
        if self.k_init_ == 'fft':
            self.FFTraversal()
            for i, c in self.centroids_.items():
                C.append(c)
        elif self.k_init_ == 'random':
            C = np.random.uniform(low=np.min(self.data, axis=0) * .4,
                                  high=np.max(self.data, axis=0) * .4,
                                  size=(self.n_clusters, self.data.shape[1]))
#             self.labels_ = []
#             for i in range(len(self.data)):
#                 self.labels_.append(np.int64(np.random.choice(range(self.n_clusters))))
#             for i in range(self.n_clusters):
#                 cs = self.data[np.where(self.labels_ == np.int64(i))[0]]
#                 if len(cs) > 0:
#                     C.append(np.mean(cs, axis=0))
        else:
            raise Exception(f"k_init={self.k_init_} is not defined")
            
        C = np.array(C)
        error = np.inf
        while error != 0:
            self.labels_ = []
            C_old = deepcopy(C)
            for i in range(len(self.data)):
                distances = [(self.distance_function(c, self.data[i]), j, self.data[i])
                    for j, c in enumerate(C)]
                distances.sort()
                cluster = distances[0][1]
                self.labels_.append(cluster)
            for i in range(self.n_clusters):
                cs = self.data[np.where(self.labels_ == np.int64(i))[0]]
                if len(cs) > 0:
                    C[i] = np.mean(cs, axis=0)
            error = np.sum(C - C_old)
        self.centroids_ = {}
        for i, c in enumerate(C):
            self.centroids_[np.int32(i)] = c
        self._inertia()
        return self

    def FFTraversal(self):
        self.centroids_ = {}
        c1_idx = np.random.choice(self.data.shape[0])
        self.centroids_[np.int32(c1_idx)] = self.data[c1_idx, :]
        while len(self.centroids_.keys()) < self.n_clusters:
            self._assign_nearest_centroids()
            self.dists_.sort()
            cn_idx=self.dists_[-1][1]
            self.centroids_[np.int32(cn_idx)] = self.data[cn_idx, :]
        self._assign_nearest_centroids()
#         self.dists_.sort()
        self._inertia()

In [ ]:
gc.collect()
import sys
local_vars = list(locals().items())
tot=0
vars_=[]
for var, obj in local_vars:
    mem=sys.getsizeof(obj)
    tot+=mem
    vars_.append((mem,var))
print("Total =",tot)
vars_.sort(reverse=True)
print(vars_)